In [90]:
import gradio as gr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [92]:
def load_data():
    df = pd.read_csv('stock_data.csv')  # Ensure correct file path
    df.columns = df.columns.str.strip()
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df[-30:]  # Keep only the last 30 days

def prepare_lstm_data(df, time_steps=5):
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df[['Close_sp500']])
    X, y = [], []
    for i in range(len(df_scaled) - time_steps):
        X.append(df_scaled[i:i+time_steps])
        y.append(df_scaled[i+time_steps])
    return np.array(X), np.array(y), scaler

def build_lstm_model(time_steps):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(time_steps, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_models():
    df = load_data()
    time_steps = 5
    
    # LSTM Model Training
    X_lstm, y_lstm, scaler = prepare_lstm_data(df, time_steps)
    if len(X_lstm) > 0:
        model_lstm = build_lstm_model(time_steps)
        model_lstm.fit(X_lstm, y_lstm, epochs=10, batch_size=4, verbose=1)
        y_lstm_pred = model_lstm.predict(X_lstm)
        y_lstm_pred = scaler.inverse_transform(y_lstm_pred)
        y_lstm_true = scaler.inverse_transform(y_lstm)
    else:
        model_lstm, y_lstm_pred, y_lstm_true = None, None, None
    
    # Linear Regression Training
    df['Lag_Close_sp500'] = df['Close_sp500'].shift(1)
    df.dropna(inplace=True)
    if len(df) > 1:
        X_lr, y_lr = df[['Lag_Close_sp500']].values, df['Close_sp500'].values
        model_lr = LinearRegression()
        model_lr.fit(X_lr, y_lr)
        y_lr_pred = model_lr.predict(X_lr)
    else:
        model_lr, y_lr_pred = None, None
    
    return model_lstm, model_lr, scaler, df, y_lstm_true, y_lstm_pred, y_lr_pred

model_lstm, model_lr, scaler, df, y_lstm_true, y_lstm_pred, y_lr_pred = train_models()

def plot_last_30_days():
    plt.figure(figsize=(10, 5))
    last_30_days = df.index[-len(y_lstm_true):]
    plt.plot(last_30_days, y_lstm_true, label="Actual Price", color="blue")
    if y_lstm_pred is not None:
        plt.plot(last_30_days, y_lstm_pred, label="LSTM Predicted Price", color="orange")
    if y_lr_pred is not None:
        plt.plot(last_30_days, y_lr_pred[-len(last_30_days):], label="Linear Regression Predicted Price", color="green")
    plt.xlabel("Date")
    plt.ylabel("Stock Price")
    plt.title("S&P 500 Stock Price Prediction")  # Removed R² references
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid()
    plt.savefig("last_30_days_plot.png")
    return "last_30_days_plot.png", ""

def get_prediction(model_choice, selected_date):
    last_30_days = df.index[-len(y_lstm_true):]
    if pd.to_datetime(selected_date) not in last_30_days:
        return "Invalid date selected. Please choose a date from the last 30 days."
    
    index = list(last_30_days).index(pd.to_datetime(selected_date))
    actual_price = y_lstm_true[index] if index < len(y_lstm_true) else "N/A"
    
    if model_choice == "LSTM" and y_lstm_pred is not None:
        predicted_price = y_lstm_pred[index] if index < len(y_lstm_pred) else "N/A"
    elif model_choice == "Linear Regression" and y_lr_pred is not None:
        predicted_price = y_lr_pred[index] if index < len(y_lr_pred) else "N/A"
    else:
        return "Prediction not available for the selected model."
    
    return f"Actual Price: {actual_price}, Predicted Price ({model_choice}): {predicted_price}"

demo = gr.Blocks()
with demo:
    gr.Markdown("""## Stock Price Prediction
    Compare the performance of LSTM and Linear Regression models for S&P 500 predictions.
    """)
    gr.Image(plot_last_30_days()[0], label="Default Prediction Plot (Last 30 Days)")
    
    model_choice = gr.Dropdown(choices=["LSTM", "Linear Regression"], label="Select Model")
    date_selection = gr.Dropdown(choices=[str(d.date()) for d in df.index[-len(y_lstm_true):]], label="Select Date")
    prediction_output = gr.Textbox(label="Prediction Result")
    
    gr.Interface(
        fn=get_prediction,
        inputs=[model_choice, date_selection],
        outputs=prediction_output,
        allow_flagging="never"
    )

demo.launch(share=True)


C:\Users\Mimy\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 0.3794
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1877
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0655
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0644
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0334
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0425
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0510
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0301
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0313
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0276
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


C:\Users\Mimy\anaconda3\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Missing file: C:\Users\Mimy\anaconda3\Lib\site-packages\gradio\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: C:\Users\Mimy\anaconda3\Lib\site-packages\gradio


In [72]:
pip install pyngrok

Note: you may need to restart the kernel to use updated packages.


In [80]:
demo.launch(server_name='10.0.0.194', server_port=7860)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.


In [86]:
!pip install pyngrok